In [1]:
spark._jsc.hadoopConfiguration().set("fs.s3.useRequesterPaysHeader","true")

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1620738130541_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
# Q1

from functools import reduce

errors = [1,10,45,3,5,40]

reduce(lambda s1, s2: s1+s2, map(lambda e: e**2, errors))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

3760

In [3]:
# Q2a

from functools import reduce

errors = [1,10,45,3,5,40]

def weighted_mean(obj1, obj2):
  w = obj1["w"] + obj2["w"]
  return ({
      "m" : (obj1["m"] * obj1["w"] + obj2["m"] * obj2["w"]) / w,
      "w" : w
  })
squared_errors = map(lambda e: {"m" : e**2, "w" : 1}, errors)
reduce( weighted_mean, squared_errors )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'m': 626.6666666666666, 'w': 6}

In [4]:
# Q2b

from functools import reduce

errors = [1,10,45,3,5,40]

def sum_and_n(obj1, obj2) :
  return {
    "sum" : obj1["sum"] + obj2["sum"],
    "n"   : obj1["n"] + obj2["n"]
  }

squared_errors = map(lambda e: {"sum" : e**2, "n" : 1}, errors)
total_sum_and_n = reduce( sum_and_n, squared_errors )
total_sum_and_n["sum"]/total_sum_and_n["n"]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

626.6666666666666

In [53]:
# A2

from pyspark.sql.types import FloatType, IntegerType, DateType
from pyspark.sql.functions import substring, col, regexp_replace

# listings_raw = spark.read.csv("s3n://spark-lab-input-data-ensai20202021/airbnb/data-insideairbnb-australia-vic-barwon-south-west-vic-2020-05-26-data-listings.csv.gz", header=True, multiLine=True, escape='"')
# listings_raw = spark.read.csv("s3n://spark-lab-input-data-ensai20202021/airbnb/", header=True, multiLine=True, escape='"')
listings_raw = spark.read.csv("s3n://spark-lab-input-data-ensai20202021/airbnb/", header=True, multiLine=True, escape='"')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [57]:
listings_raw.select("price").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+
| price|
+------+
| 833.0|
|1201.0|
| 600.0|
| 600.0|
| 410.0|
| 551.0|
| 600.0|
| 403.0|
| 742.0|
| 417.0|
| 551.0|
| 438.0|
| 953.0|
| 439.0|
| 480.0|
| 551.0|
| 418.0|
|8002.0|
| 417.0|
| 417.0|
+------+
only showing top 20 rows

In [62]:
listings = (listings_raw
  .withColumn("beds",     listings_raw["beds"    ].cast(IntegerType()))
  .withColumn("bedrooms", listings_raw["bedrooms"].cast(IntegerType()))
  .withColumn("time", listings_raw["last_scraped"].cast(DateType()))
  .withColumn("price", regexp_replace('price', '[$\\,]', '').cast(FloatType()))
  .select("id", "beds", "bedrooms", "price", "city", "time")
  .dropna()     
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [59]:
listings.groupBy("city").count().sort(col("count").desc()).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+------+
|            city| count|
+----------------+------+
|            1125|325080|
|         Beijing|105966|
|              30| 46900|
|       Barcelona| 40547|
|Ciudad de México| 38346|
|             365| 26644|
|      Washington| 17644|
|              90| 14504|
|         Beyoğlu| 14095|
|         Seattle| 13557|
|              28| 13197|
|              60| 12364|
|               7|  8788|
|              15|  8669|
|              29|  7867|
|             180|  7143|
|              31|  7051|
|           Şişli|  6570|
|           Fatih|  6539|
|          Genève|  6505|
+----------------+------+
only showing top 20 rows

In [63]:
listings.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+----+--------+------+-------+----------+
|    id|beds|bedrooms| price|   city|      time|
+------+----+--------+------+-------+----------+
| 44054|   4|       3| 833.0|Beijing|2020-05-04|
|100213|   1|       1|1201.0|Beijing|2020-05-06|
|114384|   1|       1| 600.0|Beijing|2020-05-04|
|114465|   1|       1| 600.0|Beijing|2020-05-04|
|128496|   2|       1| 410.0|Beijing|2020-04-30|
|161902|   1|       1| 551.0|Beijing|2020-05-05|
|162144|   2|       1| 600.0|Beijing|2020-05-06|
|279078|   1|       1| 403.0|Beijing|2020-05-02|
|282825|   3|       2| 742.0|Beijing|2020-05-05|
|287026|   1|       1| 417.0|Beijing|2020-05-01|
|317195|   1|       1| 551.0|Beijing|2020-05-04|
|322292|   1|       1| 438.0|Beijing|2020-05-01|
|445306|   2|       2| 953.0|Beijing|2020-05-06|
|456641|   1|       1| 439.0|Beijing|2020-04-30|
|458561|   1|       1| 480.0|Beijing|2020-05-01|
|491280|   1|       1| 551.0|Beijing|2020-05-04|
|498126|   1|       1| 418.0|Beijing|2020-04-30|
|537674|   6|       

In [64]:
# Q6

n0 = listings_raw.count()
n  = listings.count()
print("Observations before cleaning:", n0) #
print("Observations  after cleaning:", n, "(change:",  n-n0,")")     # 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Observations before cleaning: 1332873
Observations  after cleaning: 979096 (change: -353777 )

In [26]:
# Q7

print("There are", len(listings_raw.columns), "columns:")
print('\n'.join( ["- "+column for column in listings_raw.columns] ))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

There are 106 columns:
- id
- listing_url
- scrape_id
- last_scraped
- name
- summary
- space
- description
- experiences_offered
- neighborhood_overview
- notes
- transit
- access
- interaction
- house_rules
- thumbnail_url
- medium_url
- picture_url
- xl_picture_url
- host_id
- host_url
- host_name
- host_since
- host_location
- host_about
- host_response_time
- host_response_rate
- host_acceptance_rate
- host_is_superhost
- host_thumbnail_url
- host_picture_url
- host_neighbourhood
- host_listings_count
- host_total_listings_count
- host_verifications
- host_has_profile_pic
- host_identity_verified
- street
- neighbourhood
- neighbourhood_cleansed
- neighbourhood_group_cleansed
- city
- state
- zipcode
- market
- smart_location
- country_code
- country
- latitude
- longitude
- is_location_exact
- property_type
- room_type
- accommodates
- bathrooms
- bedrooms
- beds
- bed_type
- amenities
- square_feet
- price
- weekly_price
- monthly_price
- security_deposit
- cleaning_fee
- guests

In [29]:
# Q8

# 3 ways
print(listings_raw.schema)
print(listings_raw.dtypes)
listings_raw.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

StructType(List(StructField(id,StringType,true),StructField(listing_url,StringType,true),StructField(scrape_id,StringType,true),StructField(last_scraped,StringType,true),StructField(name,StringType,true),StructField(summary,StringType,true),StructField(space,StringType,true),StructField(description,StringType,true),StructField(experiences_offered,StringType,true),StructField(neighborhood_overview,StringType,true),StructField(notes,StringType,true),StructField(transit,StringType,true),StructField(access,StringType,true),StructField(interaction,StringType,true),StructField(house_rules,StringType,true),StructField(thumbnail_url,StringType,true),StructField(medium_url,StringType,true),StructField(picture_url,StringType,true),StructField(xl_picture_url,StringType,true),StructField(host_id,StringType,true),StructField(host_url,StringType,true),StructField(host_name,StringType,true),StructField(host_since,StringType,true),StructField(host_location,StringType,true),StructField(host_about,Strin

In [43]:
# Q9

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [53]:
from timeit import default_timer as t

## version 1 ------------------

start = t()

listings.summary("count", "min", "max").show()

print ("Execution time for sumary(\"count\"...):", t()-start, "s")

## version 2 ------------------

start = t()

listings.summary().show()

print ("Execution time for sumary():", t()-start, "s")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------+----+--------+-----+------------+
|summary|      id|beds|bedrooms|price|        city|
+-------+--------+----+--------+-----+------------+
|  count|    5395|5390|    5391| 5395|        5390|
|    min|10003497| 0.0|       0| 14.0|Aireys Inlet|
|    max| 9982552|30.0|      32|996.0|     Yuulong|
+-------+--------+----+--------+-----+------------+

Execution time for sumary("count"...): 7.7258904159998565 s
+-------+--------------------+------------------+------------------+------------------+------------+
|summary|                  id|              beds|          bedrooms|             price|        city|
+-------+--------------------+------------------+------------------+------------------+------------+
|  count|                5395|              5390|              5391|              5395|        5390|
|   mean|2.3919679793697868E7| 3.735250463821892|2.5572250046373584|251.14087117701575|        null|
| stddev|1.1651428098835612E7|2.2691915653686703|1.3615510149262255|14

In [54]:
# Q10

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
## version 1 ------------------

from timeit import default_timer as t
from pyspark.sql.functions import mean

start = t()

listings.filter(col("city")=="Fairhaven").select(mean('beds')).show()

print ("Execution time:", t()-start, "s")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+
|        avg(beds)|
+-----------------+
|4.724137931034483|
+-----------------+

Execution time for sumary("count"...): 2.533814053000242 s

In [7]:
## version 2 ------------------

from timeit import default_timer as t

start = t()

listings.filter(col("city")=="Fairhaven").select("beds").summary("mean").show()

print ("Execution time:", t()-start, "s")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-----------------+
|summary|             beds|
+-------+-----------------+
|   mean|4.724137931034483|
+-------+-----------------+

Execution time for sumary("count"...): 2.7298211550005362 s

In [64]:
## version 3 ------------------

from timeit import default_timer as t
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.stat    import Summarizer

start = t()

vectorizer = VectorAssembler(
    inputCols     = ["beds"],
    outputCol     = "features"
)
summarizer = Summarizer.metrics("mean")
listings_vec = vectorizer.transform(listings.filter(col("city")=="Fairhaven"))
listings_vec.select( summarizer.summary( listings_vec.features) ).show(truncate=False)


print ("Execution time:", t()-start, "s")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------------------+
|aggregate_metrics(features, 1.0)|
+--------------------------------+
|[[4.724137931034482]]           |
+--------------------------------+

Execution time for sumary("count"...): 7.656350930999906 s

In [10]:
## version 4 ---------------
sc.install_pypi_package("pandas==0.25.1")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
Package already installed for current Spark context!
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/context.py", line 1110, in install_pypi_package
    raise ValueError("Package already installed for current Spark context!")
ValueError: Package already installed for current Spark context!



In [9]:
from timeit import default_timer as t

start = t()
listings.filter(col("city")=="Fairhaven").select('beds').toPandas().mean()

print ("Execution time:", t()-start, "s")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Execution time: 0.5073776850003924 s

In [21]:
# Q11

from pyspark.sql.functions import mean
price_mean = listings.select(mean("price"))        # this is a (remote) data frame
price_mean = price_mean.first()                    # this is a (local) row
price_mean = price_mean[0]                         # this is a (local) number

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
price_mean

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

98.31562577178319

In [76]:
from pyspark.sql.functions import sum
tss = listings.withColumn("squared_error", (col("price")-price_mean)**2).select(sum("squared_error"))
tss = tss.first()
tss = tss[0]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [77]:
tss

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1628008129070.3398

In [65]:
# Q12

from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression

vectorizer = VectorAssembler( # copy-pasted from previous section...
    inputCols     = ["beds", "bedrooms"], # ... but without price
    outputCol     = "features",
    handleInvalid = 'skip'
)

listings_vec = vectorizer.transform(listings)

regressor = LinearRegression(featuresCol="features", labelCol="price")
model     = regressor.fit(listings_vec)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [66]:
model.coefficients

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DenseVector([3.7859, -1.5734])

In [67]:
model.intercept

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

263.1685687718435

In [41]:
listings_vec.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+----+--------+-----+-------+----------+------------------+---------+
|    id|beds|bedrooms|price|   city|      time|     squared_error| features|
+------+----+--------+-----+-------+----------+------------------+---------+
| 44054| 4.0|       3|833.0|Beijing|2020-05-04| 539761.1297351066|[4.0,3.0]|
|114384| 1.0|       1|600.0|Beijing|2020-05-04| 251687.2113447575|[1.0,1.0]|
|114465| 1.0|       1|600.0|Beijing|2020-05-04| 251687.2113447575|[1.0,1.0]|
|128496| 2.0|       1|410.0|Beijing|2020-04-30|  97147.1491380351|[2.0,1.0]|
|161902| 1.0|       1|551.0|Beijing|2020-05-05|204923.14267039226|[1.0,1.0]|
|162144| 2.0|       1|600.0|Beijing|2020-05-06| 251687.2113447575|[2.0,1.0]|
|279078| 1.0|       1|403.0|Beijing|2020-05-02| 92832.56789884008|[1.0,1.0]|
|282825| 3.0|       2|742.0|Beijing|2020-05-05| 414329.5736255711|[3.0,2.0]|
|287026| 1.0|       1|417.0|Beijing|2020-05-01|101559.73037723015|[1.0,1.0]|
|317195| 1.0|       1|551.0|Beijing|2020-05-04|204923.14267039226|[1.0,1.0]|

In [72]:
# Q13

dir(model)
dir(model.summary)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['__class__', '__del__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_call_java', '_create_from_java_class', '_java_obj', '_new_java_array', '_new_java_obj', 'coefficientStandardErrors', 'degreesOfFreedom', 'devianceResiduals', 'explainedVariance', 'featuresCol', 'labelCol', 'meanAbsoluteError', 'meanSquaredError', 'numInstances', 'objectiveHistory', 'pValues', 'predictionCol', 'predictions', 'r2', 'r2adj', 'residuals', 'rootMeanSquaredError', 'tValues', 'totalIterations']

In [73]:
model.summary.r2

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.0021439426844965803

In [74]:
# model.summary.explainedVariance
# je voulais recalculer le R2, mais je passe

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

3515.8229340801427

In [91]:
# Q14

from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml import Pipeline
from pyspark.sql.functions import year

vectorizer = VectorAssembler(
    inputCols     = ["beds", "bedrooms"],
    outputCol     = "features",
    handleInvalid = 'skip'
)
regressor = LinearRegression(featuresCol="features", labelCol="price")
pipeline  = Pipeline(stages = [vectorizer, regressor])

model_2021 = pipeline.fit(listings.filter(year(listings.time)==2021))
model_2020 = pipeline.fit(listings.filter(year(listings.time)==2020))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [90]:
listings.groupBy("time").count().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-----+
|      time|count|
+----------+-----+
|2021-04-04| 2737|
|2021-03-24| 2221|
|2021-02-02| 5586|
|2020-06-17| 4086|
|2021-01-24| 2727|
|2020-06-18| 5190|
|2020-07-19| 6183|
|2020-10-16| 1782|
|2021-02-10| 8620|
|2020-12-31| 1421|
|2020-10-12| 2542|
|2020-12-19|  434|
|2020-06-24|  743|
|2020-06-20|19977|
|2020-06-22| 6773|
|2020-11-08| 4982|
|2021-03-29| 2553|
|2021-02-27|11149|
|2020-11-12| 3584|
|2020-06-13| 4084|
+----------+-----+
only showing top 20 rows

In [95]:
model_2020.stages[1].coefficients

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DenseVector([5.4629, -2.2316])

In [96]:
model_2021.stages[1].coefficients

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DenseVector([0.0742, 0.0096])

In [98]:
listings.filter(year(listings.time)==2021).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+----+--------+-----+----+----------+
|   id|beds|bedrooms|price|city|      time|
+-----+----+--------+-----+----+----------+
|37125|   0|       1| 80.0| 730|2021-03-30|
|39356|   4|      37| 97.0|  28|2021-04-03|
|42127|   0|       1|100.0| 730|2021-03-30|
|43467|   0|       2|100.0| 730|2021-04-03|
|43474|   0|       1|100.0| 730|2021-04-03|
|45663|   0|       3|100.0|  30|2021-04-03|
|46483|   0|       1|100.0| 730|2021-04-01|
|50327|   0|       2|100.0| 730|2021-04-03|
|53243|   2|      80| 98.0| 365|2021-04-02|
|54499|   8|     105| 97.0|  90|2021-03-30|
|60925|   0|      73| 99.0|1125|2021-04-03|
|61987|   2|      43| 96.0| 365|2021-04-02|
|61989|   1|       9| 90.0| 730|2021-04-03|
|61991|   0|       3| 93.0| 365|2021-04-04|
|61993|   4|      14| 89.0| 730|2021-04-02|
|63364|   0|       3| 93.0| 365|2021-04-02|
|63942|   0|       5| 84.0| 365|2021-04-02|
|63957|   1|       8| 95.0| 365|2021-03-30|
|66883|   1|       3|100.0|1125|2021-04-04|
|82093|   0|      30| 98.0|  30|

In [99]:
model_2021.stages[1].intercept

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

92.84026065519177

In [100]:
model_2020.stages[1].intercept

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

325.99482824815686

In [107]:
listings.groupBy(year("time")).agg(mean("price")).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-----------------+
|year(time)|       avg(price)|
+----------+-----------------+
|      2020|315.8997442589258|
|      2021| 93.3883378470451|
+----------+-----------------+